# install

In [1]:
!pip3 install datasets==2.20.0 evaluate==0.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

# 5.4.2 Conditional Generation

### 모델

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

### 데이터셋

In [2]:
from datasets import load_dataset

dataset = load_dataset("msarmi9/korean-english-multitarget-ted-talks-task")
print(dataset)
dataset['train'][0]

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})


{'korean': '(박수) 이쪽은 Bill Lange 이고, 저는 David Gallo입니다',
 'english': "(Applause) David Gallo: This is Bill Lange. I'm Dave Gallo."}

In [3]:
tokenized_dataset = dataset.map(
    lambda batch: (
        tokenizer(
            batch["korean"],
            text_target=batch["english"],
            max_length=512,
            truncation=True,
        )
    ),
    batched=True,
    batch_size=1000,
    num_proc=2,
    remove_columns=dataset['train'].column_names,
)
tokenized_dataset['train'][0]

{'input_ids': [0,
  14338,
  10770,
  11372,
  240,
  14025,
  12471,
  12005,
  15085,
  29490,
  14676,
  24508,
  300,
  14025,
  14161,
  16530,
  15529,
  296,
  317,
  18509,
  15464,
  15585,
  20858,
  12049,
  20211,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [0,
  14338,
  264,
  311,
  311,
  17422,
  316,
  17223,
  240,
  15529,
  296,
  317,
  18509,
  15464,
  15585,
  20858,
  257,
  15054,
  303,
  15868,
  1700,
  15868,
  15085,
  29490,
  14676,
  24508,
  300,
  245,
  14943,
  238,
  308,
  15529,
  296,
  21518,
  15464,
  15585,
  20858,
  245,
  1]}

In [4]:
from transformers import DataCollatorForSeq2Seq

collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="max_length",
    max_length=512,
)
batch = collator([tokenized_dataset['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[    0, 14338, 10770,  ...,     3,     3,     3],
        [    0, 15496, 18918,  ...,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[    0, 14338,   264,  ...,  -100,  -100,  -100],
        [    0, 14603,   309,  ...,  -100,  -100,  -100]]), 'decoder_input_ids': tensor([[    1,     0, 14338,  ...,     3,     3,     3],
        [    1,     0, 14603,  ...,     3,     3,     3]])}

### 문장 생성

In [5]:
import torch

with torch.no_grad():
    logits = model(**batch).logits
logits

tensor([[[  5.4885,  18.7849,  -0.5489,  ...,   0.0465,   0.5813,  -2.2851],
         [  3.7287,  18.9676,  -1.1747,  ...,  -0.2600,  -3.4647,  -0.0973],
         [ -1.2976,   8.6322,  -5.0410,  ...,  -7.0689,  -6.1346,  -4.4141],
         ...,
         [ -9.2638,   4.4483,  -8.4506,  ..., -12.6961, -13.2626,  -7.7570],
         [ -8.4581,   4.9268,  -7.2172,  ..., -11.5651, -11.8799,  -6.8108],
         [ -8.3191,   5.2101,  -6.8817,  ..., -11.1564, -11.7052,  -6.7644]],

        [[  4.7748,  16.2666,  -3.0011,  ...,  -0.8965,  -3.3187,  -3.1041],
         [  0.6535,  19.3665,  -1.4506,  ...,   0.1562,  -4.3976,   0.1983],
         [ -5.0934,  10.8673,  -7.5637,  ...,  -6.3808,  -1.6471,  -7.2105],
         ...,
         [ -1.5132,  19.0760,   0.3272,  ...,  -2.6680,  -3.9969,   2.7315],
         [ -2.3757,  20.0047,  -0.5301,  ...,  -1.7740,  -5.1750,   0.8077],
         [ -2.2504,  19.9756,  -0.4519,  ...,  -0.6850,  -5.1072,   0.4720]]])

In [6]:
from transformers import GenerationConfig

gen_cfg = GenerationConfig(
    max_new_tokens=100,
    do_sample=True,
    temperature=1.2,
    top_k=50,
    top_p=0.95,
)
outputs = model.generate(batch['input_ids'], generation_config=gen_cfg)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])
print(model.config.eos_token_id)

것이다. (박수) 이쪽은 Bill Lange 이고, 저는 David Gallo입니다na 라면 여자는즈의즈의 아마존 가격이 순이익 아이가노트 오픈오픈 아마존 아마존 아마존 아마존 플랫폼즈의즈의파크 외식 라면lementlementlement플레이 입력 아마존 아마존 펄 펄 펄 아마존 소프트웨어 오픈하였다.[
귀포 외식 라면즈의 클라우드 서버퓨터 펄 스크린 이승엽 순매도 플랫폼 단말 개성공단 달러를 같다”고 펄 핸드 핸드 외식 외식 외식 라면 라면 농협 순매도즈의디어 프랜차이즈 외식 챔피언십 어머니의 프랜차이즈 양도 주소 아이가란드 페이지
1


# 5.4.3 Sequence Classification

### 모델

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Lay

### 데이터셋

In [3]:
from datasets import load_dataset

dataset = load_dataset("klue", "sts")

def process_data(batch):
  result = tokenizer(batch["sentence1"], text_pair=batch["sentence2"])
  result["labels"] = [x["binary-label"] for x in batch["labels"]]
  return result

tokenized_dataset = dataset.map(
    process_data,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

### 문장 생성

In [4]:
import torch
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer)
batch = collator([tokenized_dataset["train"][i] for i in range(4)])

with torch.no_grad():
  logits = model(**batch).logits

logits

tensor([[-1.3795, -0.0489],
        [-0.4536,  0.0223],
        [-0.7792,  0.2343],
        [-0.1534,  0.4338]])

### 평가 지표

In [5]:
import evaluate

f1 = evaluate.load("f1")
f1.compute(
    predictions=logits.argmax(-1),
    references=batch['labels'],
    average='micro'
)

{'f1': 0.25}

# 5.4.4 Question Answering

### 모델

In [6]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForQuestionAnswering(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNor

### 데이터셋

In [7]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
          idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
          idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset['train'].column_names
)

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Map:   0%|          | 0/17554 [00:00<?, ? examples/s]

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

### 문장 생성

In [8]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
batch = data_collator([tokenized_dataset["train"][i] for i in range(10)])
batch

{'input_ids': tensor([[    0, 14337, 26225,  ...,     3,     3,     3],
         [    0, 25092, 18001,  ..., 11270, 19903,     1],
         [    0, 25788, 13679,  ..., 19903, 15599,     1],
         ...,
         [    0, 20437, 17814,  ...,     3,     3,     3],
         [    0, 14154, 12061,  ...,     3,     3,     3],
         [    0, 14295, 14120,  ...,     3,     3,     3]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'start_positions': tensor([233,  27,   0,  78,  60,  68, 202, 319, 306, 271]),
 'end_positions': tensor([235,  29,   0,  79,  66,  74, 210, 325, 312, 275])}

In [9]:
import torch

with torch.no_grad():
  outputs = model(**batch)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = batch["input_ids"][0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400mm로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p